Dambreak with Mesh Adaptivity (2D Version)
==============================
This is a notebook that borrows heavily from the wavetank2D notebook to demonstrate the mesh adaptation feature.

Notes:
---------

- Proteus runs with logging that records verying amounts of information by setting logLevel from 1 to 11
- The log is stored  in a .log file, which can be downloaded
- If Profiling.verbose is set  to True thebn logging will show up in certain output cells of the notebook, which is usually not what you want

Define the dambreak geometry
=========================
The geometry and mesh are provided in "Dambreak.geo" and "Dambreak.msh", respectively.
The MeshAdaptPUMI class should not be initialized more than once else the kernel fails.
The following cell exposes the inputs to the MeshAdaptPUMI class that handles the loading of the model and mesh and ultimately the mesh adaptation.

In [1]:
%%writefile modelmesh.py
from proteus import Domain
import proteus.MeshTools
from proteus.MeshAdaptPUMI import MeshAdaptPUMI

nd = 2 #number of dimensions in the problem
parallelPartitioningType = proteus.MeshTools.MeshParallelPartitioningTypes.element #type of partitioning if parallel
nLayersOfOverlapForParallel = 0 #amount of ghosting if parallel
boundaries=['left','right','bottom','top'] #boundary tag dictionary
boundaryTags=dict([(key,i+1) for (i,key) in enumerate(boundaries)])

domain = Domain.PUMIDomain(dim=nd) #initialize the domain
domain.faceList=[[11],[13],[14],[12]] #model entities associated wtih boundary tags
adaptMesh = True #adapt the mesh?
adaptMesh_nSteps = 5 #amount of time steps before checking error?

hMax = 0.08
hMin = 0.00625
adaptMesh_numIter = 2 #number of iterations for mesh adaptation
errorType="ERM" #only just ERM at the moment
logSwitch="off" #on or off
target_error = 10.0 
target_element_count = 8000

domain.PUMIMesh=MeshAdaptPUMI.MeshAdaptPUMI(hmax=hMax, 
                                            hmin=hMin, 
                                            numIter=adaptMesh_numIter,
                                            sfConfig=errorType,
                                            logType=logSwitch,
                                            targetError=target_error,
                                            targetElementCount=target_element_count)

domain.PUMIMesh.loadModelAndMesh("Dambreak.null","Dambreak.msh")


Overwriting modelmesh.py


# Simulation Options

The options are controlled by the opts class. 

In [2]:
import sys
import proteus
# Required imports
from proteus.iproteus import * 
from proteus import default_n, default_s, default_so
Profiling.logLevel=5
Profiling.verbose=False

In [3]:
import dambreak_so
from proteus import Comm
from petsc4py import PETSc

so = dambreak_so
so.tnList = so.tnList
pList=[]
nList=[]
so.sList=[]
OptDB = PETSc.Options()
for (p,n) in so.pnList:
    so.sList.append(default_s)
    pList.append(__import__(p))
    reload(pList[-1])
    nList.append(__import__(n))
    reload(nList[-1])
    pList[-1].name = p
    nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
    nList[-1].levelLinearSolver = default_n.KSP_petsc4py
    OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
opts.save_dof = True
opts.dataDir='.'
opts.probDir='.'
opts.logLevel=7
opts.verbose=True

Run the simulation
==========
The simulation will be initiated by initializing the ```NumericalSolution.NS_base()``` class followed by wrapping the ```calculateSolution()``` in a thread for ease of management.


In [34]:
ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)

In [5]:
from threading import Thread
# Create a thread wrapper for the simulation.  The target must be an argument-less
# function so we wrap the call to `calculateSolution` in a simple lambda:
simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))

In [6]:
simulation_thread.start()

# Visualizing the Mesh Adaptation

At the moment, the way to visualize the mesh adaptation process is to use Paraview to view the simulation data up until the current time step. ```gatherTimes``` is a utility that gathers the data up until the current step into a separate file for viewing. For simplicity, a paraview state file is provided. Do not close the Paraview window!

In [31]:
%%script bash --bg

gatherTimes.py -f dambreak_p
paraview --state=dambreak_p.pvsm

Starting job # 3 in a separate thread.


To update the Paraview file, rerun the following cell and hit ```F5``` in Paraview to reload the file

In [28]:
%%bash
gatherTimes.py -f dambreak_p

[       0] Proteus.Profiling never initialized. Doing it at exit.
WARNING! There are options you set that were not used!
WARNING! could be spelling mistake, etc!
Option left: name:-f value: dambreak_p


If reloading the file doesn't yield any result, then the simulation has completed and you can run open the completed ```dambreak_p.xmf``` file. For simplicity, we can just rename ```dambreak_p.xmf``` to ```dambreak_p_completed.xmf``` and reload the Paraview file

In [32]:
%%bash
cp dambreak_p.xmf dambreak_p_complete.xmf